In [2]:
!pip install -q contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 6.6 MB/s eta 0:00:00


In [3]:
# Load all required library
import pandas as pd
import numpy as np
# import sys
# sys.path.append('/Users/jerawincel/anaconda3/lib/python3.10/site-packages')
import nltk

import re
from bs4 import BeautifulSoup
import sklearn
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB


In [7]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path = '/content/drive/MyDrive/CSCI544/amazon_reviews_us_Office_Products_v1_00.tsv'

In [8]:
# Read the data as a Pandas frame using Pandas
# path = '/Users/jerawincel/Documents/USC/CSCI544/HW1/amazon_reviews_us_Office_Products_v1_00.tsv'
raw_data = pd.read_table(path, on_bad_lines='skip')

new_data = raw_data[['star_rating', 'review_headline', 'review_body']].copy()
# combine review headline and review body for review
new_data.loc[:, 'review'] = new_data['review_headline'] + ' ' + new_data['review_body']
new_data = new_data[['star_rating', 'review']]

# remove emoji in reviews
def handle_emoji(string):
    if isinstance(string, str):
        emoji = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        return emoji.sub(r'', string)
    else:
        return string

new_data['review'] = new_data['review'].apply(handle_emoji)
# drop entries with null values
new_data = new_data.dropna()

new_data['class'] = 0
# Let ratings with the values of 1, 2 and 3 form class 1
new_data.loc[new_data['star_rating'].isin([1, 2, 3]), 'class'] = 1
# Let ratings with the values of 4 and 5 form class 2
new_data.loc[new_data['star_rating'].isin([4, 5]), 'class'] = 2
# Delete entries with invalid ratings
new_data = new_data[new_data['class'] != 0]

# randomly select 50000 reviews from each class
class1 = new_data[new_data['class'] == 1].sample(n = 50000, random_state=1, ignore_index=True)
class2 = new_data[new_data['class'] == 2].sample(n = 50000, random_state=1, ignore_index=True)
samples = pd.concat([class1, class2], axis=0).reset_index(drop=True)




<ipython-input-8-b2473dd718f9>:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_table(path, on_bad_lines='skip')
<ipython-input-8-b2473dd718f9>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['class'] = 0


In [9]:
# Data Cleaning
# get average length of the reviews in terms of character length before cleaning
len_before_clean = samples['review'].str.len().mean()

# convert all reviews into lowercase
X_clean = samples['review'].str.lower()
# remove the HTML and URLs from the reviews
def remove_html(review):
    if isinstance(review, str):
        soup = BeautifulSoup(review, 'html.parser')
        text = soup.get_text()
        return re.sub(r'https?://\S+|www\.\S+', '', text)
    else:
        return review

X_clean = X_clean.apply(remove_html)
# remove non-alphabetical characters
X_clean = X_clean.str.replace(r'[^a-z\s]', '',regex=True)

# remove extra spaces
def remove_spaces(string):
    return ' '.join(string.split())

X_clean = X_clean.apply(remove_spaces)
# perform contractions on the reviews
def perform_contractions(string):
    return contractions.fix(string)

X_clean = X_clean.apply(perform_contractions)

# get average length of the reviews in terms of character length after cleaning
len_after_clean = X_clean.str.len().mean()

# print average length of reviews before and after data cleaning
print(f'{len_before_clean:.2f}, {len_after_clean:.2f}')

# Data Preprocessing

# remove stop words
def remove_stop_words(review):
    token = nltk.tokenize.word_tokenize(review)
    filtered_token = [w for w in token if w not in stopwords.words('english')]
    return ' '.join(filtered_token)

X_clean = X_clean.apply(remove_stop_words)

# lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatization(review):
    token = nltk.tokenize.word_tokenize(review)
    lemmatized_words = [lemmatizer.lemmatize(w) for w in token]
    return ' '.join(lemmatized_words)

X_clean = X_clean.apply(lemmatization)

# get average length of the reviews in terms of character length after preprocessing
len_after_pre = X_clean.str.len().mean()

# print average length of reviews before and after data preprocessing
print(f'{len_after_clean:.2f}, {len_after_pre:.2f}')

<ipython-input-9-0591bb9f9f92>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(review, 'html.parser')


347.04, 329.53
329.53, 207.64


In [10]:
# split data into training and testing dataset
X_train_clean, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_clean, samples['class'], test_size=0.2, random_state=7)


In [12]:
# feature extraction
# TF-IDF Feature Extraction
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_clean)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Bag of Words (BoW) feature extraction
bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(X_train_clean)
X_test_bow = bow_vectorizer.transform(X_test)

In [13]:
# perceptron
perceptron = Perceptron(random_state=0)

# train a perceptron model using bow feature
perceptron.fit(X_train_bow, y_train)
preceptron_predict = perceptron.predict(X_test_bow)

# Report Precision, Recall, and f1-score for BoW features.
bow_accuracy = accuracy_score(y_test, preceptron_predict)
bow_precision = precision_score(y_test, preceptron_predict)
bow_recall = recall_score(y_test, preceptron_predict)
bow_f1 = f1_score(y_test, preceptron_predict)

# print('Report for training Perceptron model using BoW features:')
# print(f'Accuracy: {bow_accuracy:.2f}; Precision: {bow_precision:.2f}; Recall: {bow_recall:.2f}; F1: {bow_f1:.2f}')
print(f'{bow_precision:.3f} {bow_recall:.3f} {bow_f1:.3f}')


# train a perceptron model for tfidf
perceptron.fit(X_train_tfidf, y_train)
preceptron_predict = perceptron.predict(X_test_tfidf)

# Report Precision, Recall, and f1-score
tfidf_accuracy = accuracy_score(y_test, preceptron_predict)
tfidf_precision = precision_score(y_test, preceptron_predict)
tfidf_recall = recall_score(y_test, preceptron_predict)
tfidf_f1 = f1_score(y_test, preceptron_predict)

# print('Report for training Perceptron model using TFIDF features:')
# print(f'Accuracy: {tfidf_accuracy:.2f}; Precision: {tfidf_precision:.2f}; Recall: {tfidf_recall:.2f}; F1: {tfidf_f1:.2f}')
print(f'{tfidf_precision:.3f} {tfidf_recall:.3f} {tfidf_f1:.3f}')



0.855 0.823 0.839
0.859 0.809 0.834


In [14]:
# svm
svm_model = LinearSVC(random_state=0, max_iter=100000, tol=1e-6, dual=True)

# train svm model using bow features
svm_model.fit(X_train_bow, y_train)
svm_pred = svm_model.predict(X_test_bow)

# Report Precision, Recall, and f1-score for training SVM using BoW features.
bow_accuracy = accuracy_score(y_test, svm_pred)
bow_precision = precision_score(y_test, svm_pred)
bow_recall = recall_score(y_test, svm_pred)
bow_f1 = f1_score(y_test, svm_pred)

# print('Report for training SVM model using BoW features:')
print(f'{bow_precision:.3f} {bow_recall:.3f} {bow_f1:.3f}')

# train svm model using tfidf features
svm_model.fit(X_train_tfidf, y_train)
svm_pred = svm_model.predict(X_test_tfidf)

# Report Precision, Recall, and f1-score for training SVM using tfidf features.
tfidf_accuracy = accuracy_score(y_test, svm_pred)
tfidf_precision = precision_score(y_test, svm_pred)
tfidf_recall = recall_score(y_test, svm_pred)
tfidf_f1 = f1_score(y_test, svm_pred)

# print('Report for training SVM model using TFIDF features:')
print(f'{tfidf_precision:.3f} {tfidf_recall:.3f} {tfidf_f1:.3f}')



0.863 0.864 0.863
0.878 0.887 0.882


In [15]:
# logistic regression
logistic_model = LogisticRegression(random_state=0, max_iter=10000)

# train logistic regression model using bow features
logistic_model.fit(X_train_bow, y_train)
logistic_pred = logistic_model.predict(X_test_bow)

# Report Precision, Recall, and f1-score for training Logistic regression using BoW features.
bow_accuracy = accuracy_score(y_test, logistic_pred)
bow_precision = precision_score(y_test, logistic_pred)
bow_recall = recall_score(y_test, logistic_pred)
bow_f1 = f1_score(y_test, logistic_pred)

# print('Report for training Logistic Regression model using BoW features:')
print(f'{bow_precision:.3f} {bow_recall:.3f} {bow_f1:.3f}')

# train logistic regression model using tfidf features
logistic_model.fit(X_train_tfidf, y_train)
logistic_pred = logistic_model.predict(X_test_tfidf)

# Report Precision, Recall, and f1-score for training Logistic regression using tfidf features.
tfidf_accuracy = accuracy_score(y_test, logistic_pred)
tfidf_precision = precision_score(y_test, logistic_pred)
tfidf_recall = recall_score(y_test, logistic_pred)
tfidf_f1 = f1_score(y_test, logistic_pred)

# print('Report for training Logistic Regression model using TFIDF features:')
print(f'{tfidf_precision:.3f} {tfidf_recall:.3f} {tfidf_f1:.3f}')



0.879 0.876 0.877
0.880 0.895 0.887


In [16]:
# naive bayes
nb_model = MultinomialNB(force_alpha=True)

# train naive bayes model using bow features
nb_model.fit(X_train_bow, y_train)
nb_pred = nb_model.predict(X_test_bow)

# Report Precision, Recall, and f1-score for training naive bayes using BoW features.
bow_accuracy = accuracy_score(y_test, nb_pred)
bow_precision = precision_score(y_test, nb_pred)
bow_recall = recall_score(y_test, nb_pred)
bow_f1 = f1_score(y_test, nb_pred)

# print('Report for training Naive Bayes model using BoW features:')
print(f'{bow_precision:.3f} {bow_recall:.3f} {bow_f1:.3f}')

# train naive bayes model using bow features
nb_model.fit(X_train_tfidf, y_train)
nb_pred = nb_model.predict(X_test_tfidf)

# Report Precision, Recall, and f1-score for training naive bayes using BoW features.
tfidf_accuracy = accuracy_score(y_test, nb_pred)
tfidf_precision = precision_score(y_test, nb_pred)
tfidf_recall = recall_score(y_test, nb_pred)
tfidf_f1 = f1_score(y_test, nb_pred)

# print('Report for training Naive Bayes model using TFIDF features:')
print(f'{tfidf_precision:.3f} {tfidf_recall:.3f} {tfidf_f1:.3f}')

0.862 0.798 0.828
0.821 0.885 0.852
